In [ ]:
import pandas as pd
import openai

# Load your CSV
df = pd.read_csv("workouts_today.csv")

# Feature engineering
df["Workout_Duration_Hours"] = df["Duration_(min)"] / 60
df["Is_Running"] = (df["Activity_name"].str.lower() == "running").astype(int)
df["Is_Yoga"] = (df["Activity_name"].str.lower() == "yoga").astype(int)
df["Avg_HR_Percent_of_Max"] = df["Average_HR_(bpm)"] / df["Max_HR_(bpm)"]
df["Total_HR_Zone_Active_%"] = (
    df["HR_Zone_1 %"] + df["HR_Zone_2 %"] + df["HR_ Zone_3 %"] +
    df["HR_Zone_4 %"] + df["HR_Zone_5%"]
)
df["Energy_per_min"] = df["Energy_burned_(cal)"] / df["Duration_(min)"]

# Aggregate features
summary = {
    "Total Workouts": len(df),
    "Total Yoga Sessions": df["Is_Yoga"].sum(),
    "Total Running Sessions": df["Is_Running"].sum(),
    "Avg Duration (min)": df["Duration_(min)"].mean(),
    "Avg Activity Strain": df["Activity_Strain"].mean(),
    "Avg Energy Burned (cal)": df["Energy_burned_(cal)"].mean(),
    "Avg Max HR (bpm)": df["Max_HR_(bpm)"].mean(),
    "Avg HR % of Max": df["Avg_HR_Percent_of_Max"].mean(),
    "Avg Energy per min": df["Energy_per_min"].mean(),
}

# Save engineered features
df.to_csv("engineered_features.csv", index=False)

# Prepare summary string
summary_str = "\n".join([f"{k}: {v:.2f}" for k, v in summary.items()])

# Get user prompt
user_question = input("Enter your fitness question or goal (e.g., 'How can I improve my running performance?'): ")

# Prepare prompt for OpenAI
prompt = f"""
You are a fitness coach. Here are summary statistics from a user's recent workouts:

{summary_str}

User's question/goal: {user_question}

Based on these, provide a concise, actionable answer.
"""

# Set your OpenAI API key
openai.api_key = ""
# Get recommendation from OpenAI
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful fitness coach."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=300
)

print("LLM-Generated Fitness Recommendation:\n")
print(response.choices[0].message.content)

Enter your fitness question or goal (e.g., 'How can I improve my running performance?'):  summarise my running


LLM-Generated Fitness Recommendation:

Based on your recent workout data, you've completed a total of 37 running sessions. Through these sessions, you're averaging a duration of approximately 56.86 minutes. During these running workouts, your max heart rate is approximately 168.49 beats per minute, indicating that you are pushing your cardiovascular fitness to its peak. The strain from your activity totals around 7.8 and you're burning an average of 241.15 calories per session. This means that you average about 6.64 calories burned per minute. 

To enhance your running capacity further or, at least, to maintain your current performance, consider incorporating interval training or hill sessions, if not already included in your regimen. This can help improve your max heart rate. Additionally, try to aim for a balanced meal post-run, which can help recover muscles and replenish energy more effectively. Keep up the impressive work!


In [ ]:
import pandas as pd
import openai

# Load your data (tab-separated, adjust if needed)
df = pd.read_csv('physiological_cycles_today.csv', sep=',')
df.columns = df.columns.str.strip()  # Remove extra spaces

# Calculate summary statistics for key metrics
summary = {
    "Recovery_score_": df["Recovery_score_"].describe(),
    "Resting_heart_rate_(bpm)": df["Resting_heart_rate_(bpm)"].describe(),
    "Heart_rate_variability_(ms)": df["Heart_rate_variability_(ms)"].describe(),
    "Sleep_performance_": df["Sleep_performance_"].describe(),
    "Asleep_duration_(min)": df["Asleep_duration_(min)"].describe(),
    "Sleep_efficiency_": df["Sleep_efficiency_"].describe(),
    "Sleep_consistency_": df["Sleep_consistency_"].describe(),
    "Day_Strain": df["Day_Strain"].describe(),
    "Energy_burned_(cal)": df["Energy_burned_(cal)"].describe(),
}

low_recovery = df[df["Recovery_score_"] < 50][["Cycle_start_time", "Recovery_score_"]]
high_sleep_debt = df[df["Sleep_debt_(min)"] > 100][["Cycle_start_time", "Sleep_debt_(min)"]]

openai.api_key = ''

summary_stats = {k: v.to_dict() for k, v in summary.items()}
low_recovery_str = low_recovery.to_string(index=False)
high_sleep_debt_str = high_sleep_debt.to_string(index=False)

while True:
    user_prompt = input("Enter your question or instruction for the LLM (type 'q' to quit): ")
    if user_prompt.strip().lower() == 'q':
        print("Exiting. Goodbye!")
        break

    prompt = f"""
You are a health data analyst. Here are summary statistics for a user's health and sleep data:

Summary statistics:
{summary_stats}

Days with low recovery (<50):
{low_recovery_str}

Days with high sleep debt (>100 min):
{high_sleep_debt_str}

{user_prompt}
"""

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful health data analyst."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400
    )

    print("\nLLM-Generated Insightful Summary:\n")
    print(response.choices[0].message.content)

Enter your question or instruction for the LLM (type 'q' to quit):  do i need weight training 



LLM-Generated Insightful Summary:

As a health data analyst, I can say that your decision to incorporate weight training into your regime may depend on your personal health and fitness goals, which are not detailed in the data you provided. 

Weight training has several potential benefits like increasing strength, muscle mass, bone density, and metabolic rate, supporting weight loss and overall fitness. However, one should always consult with a healthcare provider before starting a new physical training regimen, particularly if existing health conditions are present.

Working on improving cardiovascular health and achieving sound, consistent sleep is also critical and can be inferred from the data you provided. For instance, your mean resting heart rate is relatively high, suggesting you might want to include more cardio exercise in your routine. Also, days with high sleep debt might be adversely impacting your recovery and overall health. Balancing your workouts with sufficient rest 